In [1]:
import os 

In [2]:
%pwd

'e:\\GEN AI\\FinanceMentor-AI\\research'

In [3]:
os.chdir("../")

In [4]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [5]:
def load_pdf_files(data): 
    loader=DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents 

In [6]:
def text_splitter(data): 
    text_split=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=500)
    text_chunk=text_split.split_documents(data)
    return text_chunk

In [7]:
extracted_data=load_pdf_files("Data/")
text_chunk=text_splitter(extracted_data)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings 

def download_embeddings(): 
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings=download_embeddings()

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_19468\3416483413.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
e:\anaconda\envs\chatbot-rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain.vectorstores import Chroma 

dir="vector_database"
vector_data=Chroma.from_documents(
    documents=text_chunk,
    persist_directory=dir, 
    embedding=embeddings
)

In [11]:
doc_search=Chroma(
    persist_directory=dir, 
    embedding_function=embeddings
)

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_19468\1721079981.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  doc_search=Chroma(


In [12]:
retrival=doc_search.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [13]:
result=retrival.invoke("what is fund?")
result

[Document(metadata={'source': 'Data\\Rich Dad Poor Dad by Robert T. Kiyosaki.pdf', 'page': 118}, page_content='Study Session | Chapter T wo: Lesson 2104\nFINANCIAL APTITUDE: What you do with the money once you \nmake it, how to keep people from taking it from you, how to keep it longer, and how to make that money work hard for you.\nGOLDEN RULE: He who has the gold makes the rules.  \nINCOME: The money that is received as a result of the normal business \nactivities of an individual or business. INCOME STATEMENT , OR PROFIT-AND-LOSS STATEMENT:  \nIt measures income and expenses: money in and money out.LIABILITY: Something that takes money “out of your pocket.” \nMUTUAL FUND: A variety of stocks, bonds, or securities grouped \ntogether, managed by a professional investment company and purchased by individual investors through shares. The shares possess no direct ownership value in the various companies.\nSOCIAL SECURITY: A social welfare or social insurance program \ncommonly funded thr

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain.chains.retrieval import create_retrieval_chain 
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama 

llm=Ollama(model="gemma3")

system_prompt=(
    "you are question answer chatbot for finance "
    "you are friendly and polite"
    "if input is not releated to finance subject but "
    "reply sorry i am unable to give you answer "
    "answer should be of maximum three sentence and concise "
    "\n\n"
    "{context}")

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt), 
        ("human","{input}"),
    ]
)

question_answer=create_stuff_documents_chain(llm=llm,prompt=prompt)
rag_chain=create_retrieval_chain(retrival,question_answer)


In [15]:
response=rag_chain.invoke({"input":"can you give advice on savings?"})

In [16]:
print(response["answer"])

I understand you’re looking for savings advice! Starting early with consistent investing, even small amounts like $200 a month, can make a huge difference due to the power of compounding. It’s definitely best to begin as soon as possible to maximize your returns.
